In [10]:
# Load the extension that allows us to compile CUDA code in python notebooks
# Documentation is here: https://nvcc4jupyter.readthedocs.io/en/latest/
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4oixn5a_
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4oixn5a_
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 28f872a2f99a1b201bcd0db14fdbc5a496b9bfd7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
The nvcc4jupyter extension is already loaded. To reload it, use:
  %reload_ext nvcc4jupyter


In [11]:
%%cuda_group_save -g "source" -n "data_types.h"
/**
 * A collection of commonly used data types throughout this project.
 */
#pragma once

#include <stdint.h> // uint32_t

using element_t = uint32_t;

In [12]:
%%cuda_group_save -g "source" -n "cuda_common.h"
/**
 * Standard macros that can be useful for error checking.
 * https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__ERROR.html
 */
#pragma once

#include <cuda.h>

#define CUDA_CALL(exp)                                       \
    do {                                                     \
        cudaError res = (exp);                               \
        if(res != cudaSuccess) {                             \
            printf("Error at %s:%d\n %s\n",                  \
                __FILE__,__LINE__, cudaGetErrorString(res)); \
           exit(EXIT_FAILURE);                               \
        }                                                    \
    } while(0)

#define CHECK_ERROR(msg)                                             \
    do {                                                             \
        cudaError_t err = cudaGetLastError();                        \
        if(cudaSuccess != err) {                                     \
            printf("Error (%s) at %s:%d\n %s\n",                     \
                (msg), __FILE__, __LINE__, cudaGetErrorString(err)); \
            exit(EXIT_FAILURE);                                      \
        }                                                            \
    } while (0)

In [13]:
%%cuda_group_save -g "source" -n "data_generator.h"
/**
 * Functions for generating random input data with a fixed seed
 */
#pragma once

#include <random>  // for std::mt19937, std::uniform_int_distribution
#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a1 {

/**
 * Generates and returns a vector of random uniform data of a given length, n,
 * for any integral type. Input range will be [0, 2n].
 */
template < typename T >
std::vector< T > generate_uniform( std::size_t n )
{
    // for details of random number generation, see:
    // https://en.cppreference.com/w/cpp/numeric/random/uniform_int_distribution
    std::size_t random_seed = 20240916;  // use magic seed
    std::mt19937 rng( random_seed );     // use mersenne twister generator
    std::uniform_int_distribution<> distrib(0, 2 * n);

    std::vector< T > random_data( n ); // init array
    std::generate( std::begin( random_data )
                 , std::end  ( random_data )
                 , [ &rng, &distrib ](){ return static_cast< T >( distrib( rng ) ); });

    return random_data;
}

} // namespace a1
} // namespace csc485b

In [14]:
%%cuda_group_save -g "source" -n "algorithm_choices.h"
#pragma once

#include <vector>

#include "data_types.h"

namespace csc485b {
namespace a1 {
namespace cpu {

void run_cpu_baseline( std::vector< element_t > data, std::size_t switch_at, std::size_t n );

} // namespace cpu


namespace gpu {

void run_gpu_soln( std::vector< element_t > data, std::size_t switch_at, std::size_t n );

} // namespace gpu
} // namespace a1
} // namespace csc485b

In [289]:
%%cuda_group_save -g "source" -n "cpu_baseline.cu"
/**
 * CPU methods that the GPU should outperform.
 */

#include "algorithm_choices.h"

#include <algorithm> // std::sort()
#include <chrono>    // for timing
#include <iostream>  // std::cout, std::endl

namespace csc485b {
namespace a1      {
namespace cpu     {

/**
 * Simple solution that just sorts the whole array with a built-in sort
 * function and then resorts the last portion in the opposing order with
 * a second call to that same built-in sort function.
 */
void opposing_sort( element_t * data, std::size_t invert_at_pos, std::size_t num_elements )
{
    std::sort( data, data + num_elements, std::less< element_t >{} );
    std::sort( data + invert_at_pos, data + num_elements, std::greater< element_t >{} );
}

/**
 * Run the single-threaded CPU baseline that students are supposed to outperform
 * in order to obtain higher grades on this assignment. Times the execution and
 * prints to the standard output (e.g., the screen) that "wall time." Note that
 * the functions takes the input by value so as to not perturb the original data
 * in place.
 */
void run_cpu_baseline( std::vector< element_t > data, std::size_t switch_at, std::size_t n )
{
    auto const cpu_start = std::chrono::high_resolution_clock::now();
    opposing_sort( data.data(), switch_at, n );
    auto const cpu_end = std::chrono::high_resolution_clock::now();

    std::cout << "CPU Baseline time: "
              << std::chrono::duration_cast<std::chrono::nanoseconds>(cpu_end - cpu_start).count()
              << " ns" << std::endl;

    for( auto const x : data ) std::cout << x << " "; std::cout << std::endl;
}

} // namespace cpu
} // namespace a1
} // namespace csc485b

In [288]:
%%cuda_group_save -g "source" -n "gpu_solution.cu"
/**
 * The file in which you will implement your GPU solutions!
 */

#include "algorithm_choices.h"

#include <chrono>    // for timing
#include <iostream>  // std::cout, std::endl

#include "cuda_common.h"

namespace csc485b {
namespace a1      {
namespace gpu     {

/**
 * The CPU baseline benefits from warm caches because the data was generated on
 * the CPU. Run the data through the GPU once with some arbitrary logic to
 * ensure that the GPU cache is warm too and the comparison is more fair.
 */
__global__
void warm_the_gpu( element_t * data, std::size_t invert_at_pos, std::size_t num_elements )
{
    int const th_id = blockIdx.x * blockDim.x + threadIdx.x;

    // We know this will never be true, because of the data generator logic,
    // but I doubt that the compiler will figure it out. Thus every element
    // should be read, but none of them should be modified.
    if( th_id < num_elements && data[ th_id ] > num_elements * 100 )
    {
        ++data[ th_id ]; // should not be possible.
    }
}


__global__
void opposing_sort_single_block(element_t *data, std::size_t invert_at_pos, std::size_t num_elements)
{
    extern __shared__ element_t shared_data[];

    int local_id = threadIdx.x;

    // Load data into shared memory
    shared_data[local_id] = data[local_id];

    __syncthreads();

    // Perform bitonic sort within the block
    int block_size = num_elements;
    for (int k = 2; k <= block_size; k <<= 1) {
        for (int j = k >> 1; j > 0; j >>= 1) {
            int ixj = local_id ^ j;
            if (ixj > local_id && ixj < block_size) {
                if ((local_id & k) == 0) {
                    if (shared_data[local_id] > shared_data[ixj]) {
                        // Swap
                        element_t temp = shared_data[local_id];
                        shared_data[local_id] = shared_data[ixj];
                        shared_data[ixj] = temp;
                    }
                } else {
                    if (shared_data[local_id] < shared_data[ixj]) {
                        // Swap
                        element_t temp = shared_data[local_id];
                        shared_data[local_id] = shared_data[ixj];
                        shared_data[ixj] = temp;
                    }
                }
            }
            __syncthreads();
        }
    }

    // Apply inversion directly in shared memory
    if (local_id >= invert_at_pos && local_id < (invert_at_pos + num_elements) / 2) {
        int swap_pos = num_elements - (local_id - invert_at_pos) - 1;
        if (swap_pos < num_elements) {
            element_t temp = shared_data[local_id];
            shared_data[local_id] = shared_data[swap_pos];
            shared_data[swap_pos] = temp;
        }
    }
    __syncthreads();

    // Write back to global memory
    data[local_id] = shared_data[local_id];
}

__global__
void opposing_sort(element_t *data, std::size_t num_elements)
{
    extern __shared__ element_t shared_data[];

    int th_id = blockIdx.x * blockDim.x + threadIdx.x;
    int local_id = threadIdx.x;
    int block_size = blockDim.x;

    // Load data into shared memory
    if (th_id < num_elements) {
        shared_data[local_id] = data[th_id];
    } else {
        shared_data[local_id] = 1e9; // Padding value
    }

    __syncthreads();

    // Perform bitonic sort within the block
    for (int k = 2; k <= block_size; k <<= 1) {
        for (int j = k >> 1; j > 0; j >>= 1) {
            int ixj = local_id ^ j;
            if (ixj > local_id) {
                if ((local_id & k) == 0) {
                    if (shared_data[local_id] > shared_data[ixj]) {
                        // Swap
                        element_t temp = shared_data[local_id];
                        shared_data[local_id] = shared_data[ixj];
                        shared_data[ixj] = temp;
                    }
                } else {
                    if (shared_data[local_id] < shared_data[ixj]) {
                        // Swap
                        element_t temp = shared_data[local_id];
                        shared_data[local_id] = shared_data[ixj];
                        shared_data[ixj] = temp;
                    }
                }
            }
            __syncthreads();
        }
    }

    // Write back to global memory
    if (th_id < num_elements) {
        data[th_id] = shared_data[local_id];
    }
}

__global__
void bitonic_sort_global_kernel(element_t *data, std::size_t num_elements, int j, int k)
{
    unsigned int tid = threadIdx.x + blockDim.x * blockIdx.x;
    unsigned int ixj = tid ^ j;

    if (ixj > tid && ixj < num_elements && tid < num_elements) {
        if ((tid & k) == 0) {
            // Ascending order
            if (data[tid] > data[ixj]) {
                element_t temp = data[tid];
                data[tid] = data[ixj];
                data[ixj] = temp;
            }
        } else {
            // Descending order
            if (data[tid] < data[ixj]) {
                element_t temp = data[tid];
                data[tid] = data[ixj];
                data[ixj] = temp;
            }
        }
    }
}

__global__
void reverse_last_quarter(element_t *data, std::size_t invert_at_pos, std::size_t num_elements)
{
    unsigned int tid = threadIdx.x + blockIdx.x * blockDim.x;
    unsigned int mirror_pos = num_elements - (tid - invert_at_pos) - 1;

    if (tid >= invert_at_pos && tid < (invert_at_pos + num_elements) / 2 && mirror_pos < num_elements) {
        // Swap elements
        element_t temp = data[tid];
        data[tid] = data[mirror_pos];
        data[mirror_pos] = temp;
    }
}

/**
 * Performs all the logic of allocating device vectors and copying host/input
 * vectors to the device. Times the execution of the kernels and handles the
 * inversion logic.
 */
void run_gpu_soln(std::vector<element_t> data, std::size_t invert_at_pos, std::size_t n)
{
    // Determine if data size fits within a single block
    std::size_t threads_per_block = 1024;
    bool use_single_block = (n <= threads_per_block);

    // Calculate the next power of two if needed
    std::size_t padded_size = n;
    if (!use_single_block && (n & (n - 1)) != 0) {
        padded_size = 1 << (std::size_t)ceil(log2((double)n));
        // Pad the data with maximum values
        data.resize(padded_size, std::numeric_limits<element_t>::max());
    }

    std::size_t num_elements = padded_size;
    std::size_t num_blocks = (num_elements + threads_per_block - 1) / threads_per_block;

    // Allocate arrays on the device/GPU
    element_t *d_data;
    cudaMalloc((void**)&d_data, sizeof(element_t) * num_elements);
    CHECK_ERROR("Allocating input array on device");

    // Copy the input from the host to the device/GPU
    cudaMemcpy(d_data, data.data(), sizeof(element_t) * num_elements, cudaMemcpyHostToDevice);
    CHECK_ERROR("Copying input array to device");

    // Warm the cache on the GPU for a more fair comparison
    warm_the_gpu<<<num_blocks, threads_per_block>>>(d_data, invert_at_pos, num_elements);

    // Time the execution of the kernels
    auto const kernel_start = std::chrono::high_resolution_clock::now();

    if (use_single_block) {
        opposing_sort_single_block<<<1, n, n * sizeof(element_t)>>>(d_data, invert_at_pos, n);
    } else {
        opposing_sort<<<num_blocks, threads_per_block, threads_per_block * sizeof(element_t)>>>(d_data, num_elements);

        int stages = ceil(log2((double)num_elements));
        for (int k = 2; k <= (1 << stages); k <<= 1) {
            for (int j = k >> 1; j > 0; j >>= 1) {
                bitonic_sort_global_kernel<<<num_blocks, threads_per_block>>>(d_data, num_elements, j, k);
            }
        }

        reverse_last_quarter<<<num_blocks, threads_per_block>>>(d_data, invert_at_pos, num_elements);
    }

    auto const kernel_end = std::chrono::high_resolution_clock::now();
    CHECK_ERROR("Executing kernels on device");

    // Copy the result back to the host
    cudaMemcpy(data.data(), d_data, sizeof(element_t) * n, cudaMemcpyDeviceToHost);
    CHECK_ERROR("Transferring result back to host");
    cudaFree(d_data);
    CHECK_ERROR("Freeing device memory");

    // Remove padding if any
    data.resize(n);

    std::cout << "GPU Solution time: "
              << std::chrono::duration_cast<std::chrono::nanoseconds>(kernel_end - kernel_start).count()
              << " ns" << std::endl;

    for (auto const x : data) std::cout << x << " ";
    std::cout << std::endl;
}

} // namespace gpu
} // namespace a1
} // namespace csc485b

In [287]:
%%cuda_group_save -g "source" -n "main.cu"
/**
 * Driver for the benchmark comparison. Generates random data,
 * runs the CPU baseline, and then runs your code.
 */

#include <cstddef>  // std::size_t type
#include <iostream> // std::cout, std::endl
#include <vector>

#include "algorithm_choices.h"
#include "data_generator.h"
#include "data_types.h"
#include "cuda_common.h"

int main()
{
    std::size_t const n = 1 << 15;
    std::size_t const switch_at = 3 * ( n >> 2 ) ;

    auto data = csc485b::a1::generate_uniform< element_t >( n );
    csc485b::a1::cpu::run_cpu_baseline( data, switch_at, n );
    csc485b::a1::gpu::run_gpu_soln( data, switch_at, n );

    return EXIT_SUCCESS;
}

In [290]:
%cuda_group_run --group "source" --compiler-args "-O3 -g -std=c++20 -arch=sm_75"

CPU Baseline time: 2194742 ns
3 9 20 24 27 28 28 28 29 30 32 32 33 33 36 37 38 39 40 44 45 46 49 50 53 55 58 60 62 67 68 68 69 71 72 73 74 77 80 82 87 90 98 100 101 102 103 106 107 109 109 109 112 114 115 119 123 124 124 125 125 127 129 130 133 135 141 142 147 147 148 149 150 153 156 160 163 165 170 171 172 172 174 178 179 179 181 183 183 185 186 187 188 189 190 192 193 194 194 195 197 197 198 200 201 201 202 202 203 204 205 207 208 210 211 213 213 214 215 215 215 216 216 216 217 219 219 222 222 230 233 233 233 236 236 239 242 244 246 247 248 248 250 252 252 252 253 253 255 258 262 267 267 270 272 277 279 280 281 284 286 287 287 287 288 289 289 290 291 293 293 298 304 304 306 312 314 317 317 326 326 326 329 334 334 335 338 338 340 340 342 344 344 350 352 352 352 354 362 365 367 369 370 371 376 377 377 378 382 382 384 385 388 389 389 390 392 392 393 397 397 398 400 401 402 414 418 424 426 428 428 428 430 430 433 435 435 436 438 438 438 439 440 441 445 452 452 453 453 457 458 458 460 463